In [1]:
import spacy
import textacy
from spacy.matcher import PhraseMatcher
from textblob import TextBlob
nlp = spacy.load("en_core_web_sm")


In [28]:
# question="Where is your new cat?"
question="To whom shall I give the test results?"



In [29]:
doc = nlp(question)

In [30]:
for tok in doc:
    print(tok.text,tok.pos_, tok.lemma_, tok.tag_, tok.dep_,tok.shape_, tok.is_alpha, tok.is_stop)

To PART to TO prep Xx True True
whom PRON whom WP pobj xxxx True True
shall VERB shall MD aux xxxx True False
I PRON -PRON- PRP nsubj X True True
give VERB give VB ROOT xxxx True True
the DET the DT det xxx True True
test NOUN test NN compound xxxx True False
results NOUN result NNS dobj xxxx True False
? PUNCT ? . punct ? False False


In [31]:
text_ext = textacy.extract.subject_verb_object_triples(doc)
for item in text_ext:
    print(item)

# Detecting open-ended interogative sentences

In [2]:
matcher = PhraseMatcher(nlp.vocab, attr='LOWER') #making a pattern matcher
question_terms = ["who","whom","whose","what","when","where","why","which","how"]
patterns = [nlp(text) for text in question_terms]
matcher.add("question", None, *patterns)

In [37]:
def detect_sentences(text):
    if len(text.split()) > 0:
        text_blob = TextBlob(text)
        sentences=[]
        for sen in text_blob.sentences:
            sentences.append(str(sen))
        return sentences
    
ver_tags = ["VBZ","VB","VBD","VBG","VBN","VBP"] 

def interogation_detectore(sentence):
    doc = nlp(sentence)
    if len(matcher(doc)) !=0:
        matches = matcher(doc)
        first_filter=doc[matches[0][1]:]
        tag_list = [item.tag_ for item in first_filter]
        dep_list = [item.dep_ for item in first_filter]
        if "subj" in first_filter[0].dep_ or "obj" in first_filter[0].dep_:
            return {"question":first_filter,"subject":"will be in answer","tag":"direct-question without subject"}
            
        try:
            flag=False
            for i in range(1,len(tag_list)):
                for j in range(i+1,len(tag_list)):
                    if tag_list[i] in ver_tags and "nsubj" in dep_list[j]:
                        flag=True
                        subject_index=j
                        break
            if flag:
                subject=[]
                objects =[]
                for tok in doc:
                    if "subj" in tok.dep_:
                        subject.append(str(tok))
                    if "obj" in tok.dep_:
                        objects.append(tok)
                return {"question":first_filter,"subject":subject,"object":objects,"tag":"direct-question"}
            else:
                return "not a question"
        except IndexError:
            pass
        
    elif doc[0].pos_ == "VERB" or doc[0].tag_ in ver_tags:
        if "neg" in doc[1].dep_:
            if doc[2].tag_ in ver_tags and "nsubj" not in [item.dep_ for item in doc]:
                subject=[]
                objects =[]
                for tok in doc:
                    if "subj" in tok.dep_:
                        subject.append(str(tok))
                    if "obj" in tok.dep_:
                        objects.append(tok)
                return {"question":sentence,"subject":subject,"object":objects,"tag":"negation yes/no question"}
            if doc[2].tag_ in ver_tags and "nsubj" in [item.dep_ for item in doc]:
                subject=[]
                objects =[]
                for tok in doc:
                    if "subj" in tok.dep_:
                        subject.append(str(tok))
                    if "obj" in tok.dep_:
                        objects.append(tok)
                return {"question":sentence,"subject":str(doc[[item.dep_ for item in doc].index("nsubj")]),"objects":objects,"tag":"negation yes/no question"}
        else:
            if ("nsubj" in doc[1].dep_) or ("DET" in doc[1].pos_ and "nsubj" in doc[2].dep_):
                if "or" in [str(item) for item in doc]:
                    return {"question":sentence,"tag":"aleternative interrogative"}
                subject=[]
                objects =[]
                for tok in doc:
                    if "subj" in tok.dep_:
                        subject.append(str(tok))
                    if "obj" in tok.dep_:
                        objects.append(tok)
                return {"question":sentence,"subject":subject,"objects":objects,"tag":"yes/no question"}
        
    else:
        return "not a question"
    
    

In [51]:
example="how did the day"
interogation_detectore(example)
# detect_sentences(example)

'not a question'

In [33]:
x

{'question': Who did you give the last cookie to?,
 'subject': ['you'],
 'object': [Who, cookie],
 'tag': 'direct-question'}

# Testing

In [144]:
import pandas as pd 
import psycopg2

In [145]:
conn = psycopg2.connect(host="35.200.234.61",database="sales", user="postgres", password="cx6ac54nmgGtLD1y")

In [148]:
sql= """select
	id,
	text_,
	snippet.speaker
from
	snippet
where
	snippet.task_id in (
	select
		task.id
	from
		task
	where
		task.actor in (
		select
			org_user.userid
		from
			org_user
		where
			org_user.organizationid = 154)) order by snippet.id asc limit 100"""

In [203]:
# df = pd.read_sql_query(sql,conn)
df = pd.read_excel("/home/andy/Desktop/test_data.xlsx")

In [204]:
restul={"orignal_sentence":[],"sentences":[],"question_result":[]}
for item in df.text_.to_list():
    sentences = detect_sentences(item)
    restul["orignal_sentence"].append(item)
    restul["sentences"].append(sentences)
    q_results=[]
    for sentence in sentences:
        q_results.append(interogation_detectore(sentence))
    restul["question_result"].append(q_results)

whom are you buying this mattress for
2 nsubj
what is size do you want the mattress to be
4 nsubj
4 nsubj
7 nsubj
where do you live
2 nsubj
whom do you buy the goods
2 nsubj
how soon will you want the goods to be shipped
6 nsubjpass
What are the problems you have with the current system
3 nsubj
Which database do you currently have
3 nsubj
when do you want to buy the product
2 nsubj
what types of analysis do you conduct with the system
5 nsubj
where are you thinking of starting another centres
2 nsubj
what is your current salary
4 nsubj
what is the amount you will need
3 nsubj
how many people are working in your office
how do you hire people at present
2 nsubj
which device do you use
3 nsubj
which tools do you use for finding prospects
3 nsubj
how do you handle social media engagements
2 nsubj
what business are you in
3 nsubj
how do you meet your capital needs
2 nsubj
how do you manage high value orders
2 nsubj
where do you want the office to be
2 nsubj
5 nsubj
how many people will need

how do you manage to supply in large quantities
2 nsubj
when will you occupy the space
who will be sleeping on this regularly
what size mattress do you currently use
4 nsubj
which place are you in
3 nsubj
where do you get hold of the raw materials
2 nsubj
how quickly would you want us to transfer the materials to you
5 nsubj
what are you pain points when people work on this system
2 nsubj
how do you manage to save and retrieve the data
2 nsubj
when can we begin a new contract
what type of actionable reports are you able to generate with the current system
6 nsubj
what are the various steps in you getting the reports
4 nsubj
how we can enhance your team's performance
how much stiped do you get
4 nsubj
4 nsubj
how much do you need
3 nsubj
how many hands are there in your office
which channels do you use to fill the positions
3 nsubj
which platforms do you take help from
3 nsubj
how do you ensure that the prospects are engaged
2 nsubj
6 nsubj
what is your product line
4 nsubj
how do you m

In [205]:
new_df = pd.DataFrame.from_dict(restul)

In [206]:
new_df.to_excel("/home/andy/Desktop/check1.xlsx")